In [1]:
import pickle
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
natural_df = pd.DataFrame(columns = ["family", "family_id", "sequence_length_mean", "hamming_mean", "hmmer_mean", "ptm_mean", "plddt_mean",
                                        "sequence_length_median", "hamming_median", "hmmer_median", "ptm_median", "plddt_median",
                                        "sequence_length_std", "hamming_std", "hmmer_std", "ptm_std", "plddt_std",])

for family_idx in [20, 22, 50, 98, 100, 141, 177, 222, 233, 265, 303, 327, 338, 341, 376, 393, 471, 479, 481]:
    
    with open(f"evaluations/natural/sequence_df_{family_idx}", "rb") as f:
        sequence_df = pickle.load(f)

    eval_dict = {"family": [family_idx], "family_id": [sequence_df["family_id"].iloc[0]]}

    eval_dict["sequence_length_mean"] = [np.mean(list(sequence_df["sequence_length"]))]
    eval_dict["hamming_mean"] = [np.mean(list(sequence_df["min_hamming"]))]
    eval_dict["hmmer_mean"] = [np.mean(list(sequence_df["score_gen"]))]
    eval_dict["ptm_mean"] = [np.mean(list(sequence_df["ptm"]))]
    eval_dict["plddt_mean"] = [np.mean(list(sequence_df["mean_plddt"]))]

    eval_dict["sequence_length_median"] = [np.median(list(sequence_df["sequence_length"]))]
    eval_dict["hamming_median"] = [np.median(list(sequence_df["min_hamming"]))]
    eval_dict["hmmer_median"] = [np.median(list(sequence_df["score_gen"]))]
    eval_dict["ptm_median"] = [np.median(list(sequence_df["ptm"]))]
    eval_dict["plddt_median"] = [np.median(list(sequence_df["mean_plddt"]))]

    eval_dict["sequence_length_std"] = [np.std(list(sequence_df["sequence_length"]))]
    eval_dict["hamming_std"] = [np.std(list(sequence_df["min_hamming"]))]
    eval_dict["hmmer_std"] = [np.std(list(sequence_df["score_gen"]))]
    eval_dict["ptm_std"] = [np.std(list(sequence_df["ptm"]))]
    eval_dict["plddt_std"] = [np.std(list(sequence_df["mean_plddt"]))]

    natural_df = pd.concat([natural_df, pd.DataFrame(eval_dict)], ignore_index=True)

with open(f"evaluations/natural/final_evaluation", "wb") as f:
    pickle.dump(natural_df, f)

In [3]:
with open(f"evaluations/natural/final_evaluation", "rb") as f:
    natural_df = pickle.load(f)

In [ ]:
for model_name in ['protxlstm_102M_60B', 'protmamba_107M_195B', 'protxlstm_26M_30B', 'protmamba_28M_30B']:

    evaluation_df = pd.DataFrame(columns = ["family", "family_id", "sequence_length_mean", "hamming_mean", "hmmer_mean", "ptm_mean", "plddt_mean",
                                        "sequence_length_median", "hamming_median", "hmmer_median", "ptm_median", "plddt_median",
                                        "sequence_length_std", "hamming_std", "hmmer_std", "ptm_std", "plddt_std",
                                        "sequence_length_delta", "hamming_delta", "hmmer_delta", "ptm_delta", "plddt_delta",
                                        "sequence_length_ks", "hamming_ks", "hmmer_ks", "ptm_ks", "plddt_ks",
                                        "sequence_length_ppl_r", "hamming_ppl_r", "hmmer_ppl_r", "ptm_ppl_r", "plddt_ppl_r"
                                        ])

    for family_idx in [20, 22, 50, 98, 100, 141, 177, 222, 233, 265, 303, 327, 338, 341, 376, 393, 471, 479, 481]:

        with open(f"evaluations/{model_name}/sequence_df_{family_idx}", "rb") as f:
            sequence_df = pickle.load(f)

        with open(f"evaluations/natural/sequence_df_{family_idx}", "rb") as f:
            natural_df = pickle.load(f)

        eval_dict = {"family": [family_idx], "family_id": [sequence_df["family_id"].iloc[0]]}
        ptm_filtered_df = sequence_df.loc[sequence_df["ptm"] != 0]

        eval_dict["sequence_length_ppl_r"] = [stats.pearsonr(list(ptm_filtered_df["sequence_length"]), list(ptm_filtered_df["perplexity"])).statistic]
        eval_dict["hamming_ppl_r"] = [stats.pearsonr(list(ptm_filtered_df["min_hamming"]), list(ptm_filtered_df["perplexity"])).statistic]
        eval_dict["hmmer_ppl_r"] = [stats.pearsonr(list(ptm_filtered_df["score_gen"]), list(ptm_filtered_df["perplexity"])).statistic]
        eval_dict["ptm_ppl_r"] = [stats.pearsonr(list(ptm_filtered_df["ptm"]), list(ptm_filtered_df["perplexity"])).statistic]
        eval_dict["plddt_ppl_r"] = [stats.pearsonr(list(ptm_filtered_df["mean_plddt"]), list(ptm_filtered_df["perplexity"])).statistic]

        best_df = ptm_filtered_df.sort_values(by="perplexity")[:100]
        eval_dict["sequence_length_mean"] = [np.mean(list(best_df["sequence_length"]))]
        eval_dict["hamming_mean"] = [np.mean(list(best_df["min_hamming"]))]
        eval_dict["hmmer_mean"] = [np.mean(list(best_df["score_gen"]))]
        eval_dict["ptm_mean"] = [np.mean(list(best_df["ptm"]))]
        eval_dict["plddt_mean"] = [np.mean(list(best_df["mean_plddt"]))]

        eval_dict["sequence_length_median"] = [np.median(list(best_df["sequence_length"]))]
        eval_dict["hamming_median"] = [np.median(list(best_df["min_hamming"]))]
        eval_dict["hmmer_median"] = [np.median(list(best_df["score_gen"]))]
        eval_dict["ptm_median"] = [np.median(list(best_df["ptm"]))]
        eval_dict["plddt_median"] = [np.median(list(best_df["mean_plddt"]))]

        eval_dict["sequence_length_std"] = [np.std(list(best_df["sequence_length"]))]
        eval_dict["hamming_std"] = [np.std(list(best_df["min_hamming"]))]
        eval_dict["hmmer_std"] = [np.std(list(best_df["score_gen"]))]
        eval_dict["ptm_std"] = [np.std(list(best_df["ptm"]))]
        eval_dict["plddt_std"] = [np.std(list(best_df["mean_plddt"]))]

        eval_dict["sequence_length_delta_mean"] = [np.mean(list(best_df["sequence_length"]))-np.mean(list(natural_df["sequence_length"]))]
        eval_dict["hamming_delta_mean"] = [np.mean(list(best_df["min_hamming"]))-np.mean(list(natural_df["min_hamming"]))]
        eval_dict["hmmer_delta_mean"] = [np.mean(list(best_df["score_gen"]))-np.mean(list(natural_df["score_gen"]))]
        eval_dict["ptm_delta_mean"] = [np.mean(list(best_df["ptm"]))-np.mean(list(natural_df["ptm"]))]
        eval_dict["plddt_delta_mean"] = [np.mean(list(best_df["mean_plddt"]))-np.mean(list(natural_df["mean_plddt"]))]

        eval_dict["sequence_length_delta_median"] = [np.median(list(best_df["sequence_length"]))-np.median(list(natural_df["sequence_length"]))]
        eval_dict["hamming_delta_median"] = [np.median(list(best_df["min_hamming"]))-np.median(list(natural_df["min_hamming"]))]
        eval_dict["hmmer_delta_median"] = [np.median(list(best_df["score_gen"]))-np.median(list(natural_df["score_gen"]))]
        eval_dict["ptm_delta_median"] = [np.median(list(best_df["ptm"]))-np.median(list(natural_df["ptm"]))]
        eval_dict["plddt_delta_median"] = [np.median(list(best_df["mean_plddt"]))-np.median(list(natural_df["mean_plddt"]))]

        eval_dict["sequence_length_ks"] = [stats.kstest(list(best_df["sequence_length"]), list(natural_df["sequence_length"])).statistic]
        eval_dict["hamming_ks"] = [stats.kstest(list(best_df["min_hamming"]), list(natural_df["min_hamming"])).statistic]
        eval_dict["hmmer_ks"] = [stats.kstest(list(best_df["score_gen"]), list(natural_df["score_gen"])).statistic]
        eval_dict["ptm_ks"] = [stats.kstest(list(best_df["ptm"]), list(natural_df["ptm"])).statistic]
        eval_dict["plddt_ks"] = [stats.kstest(list(best_df["mean_plddt"]), list(natural_df["mean_plddt"])).statistic]

        eval_dict["sequence_length_p"] = [stats.kstest(list(best_df["sequence_length"]), list(natural_df["sequence_length"])).pvalue]
        eval_dict["hamming_p"] = [stats.kstest(list(best_df["min_hamming"]), list(natural_df["min_hamming"])).pvalue]
        eval_dict["hmmer_p"] = [stats.kstest(list(best_df["score_gen"]), list(natural_df["score_gen"])).pvalue]
        eval_dict["ptm_p"] = [stats.kstest(list(best_df["ptm"]), list(natural_df["ptm"])).pvalue]
        eval_dict["plddt_p"] = [stats.kstest(list(best_df["mean_plddt"]), list(natural_df["mean_plddt"])).pvalue]

        evaluation_df = pd.concat([evaluation_df, pd.DataFrame(eval_dict)], ignore_index=True)

    eval_dict

    with open(f"evaluations/{model_name}/final_evaluation", "wb") as f:
        pickle.dump(evaluation_df, f)


In [ ]:
eval_summary = pd.DataFrame(columns = ['natural', 'protxlstm_26M_30B', 'protmamba_28M_30B', 'protxlstm_102M_60B', 'protmamba_107M_195B', ], 
                            index=['sequence_length_mean', 'hamming_mean', 'hmmer_mean', 'ptm_mean', 'plddt_mean', 
        'sequence_length_median', 'hamming_median', 'hmmer_median', 'ptm_median', 'plddt_median',
        'sequence_length_delta_mean', 'hamming_delta_mean', 'hmmer_delta_mean', 'ptm_delta_mean', 'plddt_delta_mean', 
        'sequence_length_delta_median', 'hamming_delta_median', 'hmmer_delta_median', 'ptm_delta_median', 'plddt_delta_median', 
        'sequence_length_ks', 'hamming_ks', 'hmmer_ks', 'ptm_ks', 'plddt_ks', 
        'sequence_length_ppl_r', 'hamming_ppl_r', 'hmmer_ppl_r', 'ptm_ppl_r', 'plddt_ppl_r'])

for model_name in ['natural', 'protxlstm_26M_30B', 'protmamba_28M_30B', 'protxlstm_102M_60B', 'protmamba_107M_195B']:
    with open(f"evaluations/{model_name}/final_evaluation", "rb") as f:
        eval_df = pickle.load(f)

    for metric in ['sequence_length_mean', 'hamming_mean', 'hmmer_mean', 'ptm_mean', 'plddt_mean', 
        'sequence_length_median', 'hamming_median', 'hmmer_median', 'ptm_median', 'plddt_median',
        'sequence_length_delta_mean', 'hamming_delta_mean', 'hmmer_delta_mean', 'ptm_delta_mean', 'plddt_delta_mean', 
        'sequence_length_delta_median', 'hamming_delta_median', 'hmmer_delta_median', 'ptm_delta_median', 'plddt_delta_median', 
        'sequence_length_ks', 'hamming_ks', 'hmmer_ks', 'ptm_ks', 'plddt_ks', 
        'sequence_length_ppl_r', 'hamming_ppl_r', 'hmmer_ppl_r', 'ptm_ppl_r', 'plddt_ppl_r']:

        if metric in eval_df.columns:
            mean = round(np.mean([abs(x) for x in list(eval_df[metric])]), 2)
            confidence_interval = round(1.96*np.std([abs(x) for x in list(eval_df[metric])])/np.sqrt(19), 2)
            eval_summary.loc[metric, model_name] = '$' + str(mean) + '^{\pm ' + str(confidence_interval) + '}$'

display(eval_summary)

In [ ]:
models = ['natural', 'protxlstm_102M_60B', 'protmamba_107M_195B']  #, 'protxlstm_26M_30B', 'protmamba_28M_30B']
metrics = ['sequence_length', 'min_hamming', 'score_gen', 'ptm', 'mean_plddt']
datasets = [20, 22, 50, 98, 100, 141, 177, 222, 233, 265] #, 303, 327, 338, 341, 376, 393, 471, 479, 481]

model_names = {"natural": 'Natural Sequences', "protxlstm_102M_60B": "Prot-xLSTM-102M", "protmamba_107M_195B": "ProtMamba-107M", "protxlstm_26M_30B": "Prot-xLSTM-26M", "protmamba_28M_30B": "ProtMamba-28M"}
metric_names = {'sequence_length': 'Sequence Length', 'min_hamming': "Min. Hamming", 'score_gen': "HMMER score", 'ptm': "pTM", 'mean_plddt': 'pLDDT'}
dataset_names = {}

cd = {"natural": "grey", "protxlstm_102M_60B": (48/255, 115/255, 173/255, 1.0), "protmamba_107M_195B": (223/255, 137/255, 83/255, 1.0), "protxlstm_26M_30B": (48/255, 115/255, 173/255, 0.5), "protmamba_28M_30B": (223/255, 137/255, 83/255, 0.5)}

data = []

for model in models:
    for dataset in datasets:
        try:
            with open(f"evaluations/{model}/sequence_df_{dataset}", "rb") as f:
                sequence_df = pickle.load(f)
                if model != "natural":
                    sequence_df = sequence_df.loc[sequence_df["ptm"] != 0]
                    sequence_df = sequence_df.sort_values(by="perplexity")[:100]
                else: dataset_names[dataset] = sequence_df["family_id"].iloc[0]
            for metric in metrics:
                values = list(sequence_df[metric])
                for value in values:
                    data.append([model_names[model], metric, dataset, value, cd[model]])
        except:
            for metric in metrics:
                values = [0]
                for value in values:
                    data.append([model_names[model], metric, dataset, value,  cd[model]])
            print(f'No data for {model}, family {dataset}, metric {metric}.')

df = pd.DataFrame(data, columns=['model', 'metric', 'dataset', 'value', "color"])

fig, axes = plt.subplots(len(datasets), len(metrics), figsize=(12,15), sharey='col')

plt.subplots_adjust(wspace=0.3)

palette = sns.color_palette("Set2", len(models))

for j, dataset in enumerate(datasets):  # Datasets on the y-axis
    
    for i, metric in enumerate(metrics):  # Metrics on the x-axis
        ax = axes[j, i]

        subset = df[(df['metric'] == metric) & (df['dataset'] == dataset)]
        
        sns.boxplot(x='model', y='value', data=subset, palette=list(cd.values()), hue='model', legend=False, ax=ax, )
        
        for k, patch in enumerate(ax.patches):
            if k in [3,4]:
                r, g, b, a = patch.get_facecolor()
                patch.set_facecolor((r, g, b, .5))

        ax.set_xticks([]) 
        ax.set_xlabel(None) 
        ax.set_ylabel(None) 
        if j == 0:
            ax.set_title(metric_names[metric])  # Set metric name as the title for the column
        if i == 0:
            ax.set_ylabel(dataset_names[dataset])  # Set dataset name as the ylabel for the row

handles = [plt.Line2D([0], [0], color=list(cd.values())[k], lw=5) for k in range(len(models))]


fig.legend(handles, list(model_names.values()), loc='upper center', title='Models', ncol=len(models), bbox_to_anchor=(0.5, 1.035))

plt.tight_layout()
plt.savefig(f'figures/generated_sequences_scores.pdf', format='pdf', dpi=1200, transparent=True,  bbox_inches='tight')

plt.show()